<a href="https://colab.research.google.com/github/veekaybee/viberary/blob/bert_notebook/src/notebooks/08_bert_sentence_transformer_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys

!{sys.executable} -W ignore:DEPRECATION -m pip install --quiet duckdb==0.7.1 \
duckdb-engine \
watermark \
jupysql \
sqlalchemy \
python-snappy \
pyarrow \
memray \
pandas  \
ipywidgets  \
matplotlib \
gensim \
nltk \
plotly \
redis==4.5.3 \
jupyter-black \
sentence_transformers \
redis \
jupyter_black

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.6/238.6 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.8/271.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 123.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.7 MB/s eta 0:00:0

In [3]:
# Autoformat cells on run
import jupyter_black
import pandas as pd

jupyter_black.load()

In [4]:

import re
import pandas as pd
import string
from time import time

In [5]:
# set log level for model training
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(
    format="%(levelname)s - %(asctime)s: %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
embeddings = pd.read_csv("/content/drive/My Drive/Colab Notebooks/embedding_data.csv")

In [8]:
sample = embeddings[:50]
sample

,Unnamed: 0,title,sentence
0,0,W.C. Fields: A Life on Film,w c fields a life on film
1,1,Good Harbor,good harbor anita diamant s international best...
2,2,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",the unschooled wizard sun wolf and starhawk ...
3,3,Best Friends Forever,best friends forever addie downs and valerie a...
4,4,Runic Astrology: Starcraft and Timekeeping in ...,runic astrology starcraft and timekeeping in ...
5,5,The Aeneid for Boys and Girls,the aeneid for boys and girls relates in vigor...
6,6,The Wanting of Levine,the wanting of levine
7,7,All's Fairy in Love and War (Avalon: Web of Ma...,all s fairy in love and war avalon web of ma...
8,8,Playmaker: A Venom Series Novella,playmaker a venom series novella secrets som...
9,9,The Devil's Notebook,the devil s notebook wisdom humor and dark o...


In [9]:
corpus = sample["sentence"].tolist()
titles = sample["title"].tolist()
indices = sample.index.tolist()

In [10]:
from sentence_transformers import SentenceTransformer, util

#  A common value for BERT & Co. are 512 word pieces, which correspond to about 300-400 words (for English).
# Longer texts than this are truncated to the first x word pieces.
# By default, the provided methods use a limit fo 128 word pieces, longer inputs will be truncated
# the runtime and the memory requirement grows quadratic with the input length - we'll have to play around with this

# Change the length to 200
model = SentenceTransformer("all-MiniLM-L6-v2")
model.max_seq_length = 200


corpus_embeddings = model.encode(
    corpus, show_progress_bar=True,device='cuda', convert_to_numpy=False
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# Output embeddings

embeddings_list = [x.tolist() for x in corpus_embeddings]

embedding_tuple = list(zip(titles,indices, embeddings_list))

for title, index,embedding in embedding_tuple:
    print("Sentence:", title)
    print("Index", index)
    print("Embedding:", embedding )
    print("")

Sentence: W.C. Fields: A Life on Film
Index 0
Embedding: [-0.09234858304262161, -0.006951860152184963, -0.059966497123241425, -0.009789649397134781, 0.03234785795211792, 0.0607537180185318, 0.08125867694616318, 0.04322823882102966, 0.05158371105790138, -0.014283844269812107, 0.02573598176240921, -0.03713681176304817, -0.038165949285030365, 0.07240445911884308, -0.08822835981845856, 0.05091804265975952, -0.035940345376729965, -0.008459091186523438, -0.035312194377183914, 0.025696834549307823, -0.050433337688446045, -0.029636410996317863, 0.008075189776718616, 0.01074928231537342, -0.08982584625482559, -0.001277444651350379, -0.03546484559774399, 0.0639660656452179, -0.07411990314722061, -0.05810193344950676, -0.04055662080645561, 0.14181599020957947, 0.012416965328156948, 0.028205057606101036, 0.04940115660429001, 0.0382041372358799, -0.0006648478447459638, 0.004045507404953241, -0.03291958197951317, 0.018863985314965248, -0.09515189379453659, -0.08359432965517044, 0.03893107548356056, 

In [19]:
from tqdm import tqdm
import numpy as np
import pyarrow as pa
import pandas as pd

df = pd.DataFrame(embedding_tuple, columns=['sentence', 'index', 'embeddings'])

fields = [
    ('sentence', pa.string()),
    ('index', pa.int64(), False),
    ('embeddings', pa.large_list(pa.float64()))
]
schema = pa.schema(fields)

df.to_parquet('embeddings.snappy', engine='pyarrow', compression='snappy',schema=schema)

In [22]:
pqt = pd.read_parquet('embeddings.snappy')

In [24]:
import numpy as np

embedding_dict = dict(
    zip(
        pqt["index"],
        pqt["embeddings"]
    )
)

In [25]:

!curl -fsSL https://packages.redis.io/redis-stack/redis-stack-server-6.2.6-v7.focal.x86_64.tar.gz -o redis-stack-server.tar.gz
!tar -xvf redis-stack-server.tar.gz
!pip install redis

!./redis-stack-server-6.2.6-v7/bin/redis-stack-server --daemonize yes




./
./redis-stack-server-6.2.6-v7/
./redis-stack-server-6.2.6-v7/bin/
./redis-stack-server-6.2.6-v7/bin/redis-benchmark
./redis-stack-server-6.2.6-v7/bin/redis-cli
./redis-stack-server-6.2.6-v7/bin/redis-sentinel
./redis-stack-server-6.2.6-v7/bin/redis-stack-server
./redis-stack-server-6.2.6-v7/bin/redis-check-rdb
./redis-stack-server-6.2.6-v7/bin/redis-check-aof
./redis-stack-server-6.2.6-v7/bin/redis-server
./redis-stack-server-6.2.6-v7/share/
./redis-stack-server-6.2.6-v7/share/RSAL_LICENSE
./redis-stack-server-6.2.6-v7/share/APACHE_LICENSE
./redis-stack-server-6.2.6-v7/lib/
./redis-stack-server-6.2.6-v7/lib/redisgraph.so
./redis-stack-server-6.2.6-v7/lib/redistimeseries.so
./redis-stack-server-6.2.6-v7/lib/rejson.so
./redis-stack-server-6.2.6-v7/lib/redisbloom.so
./redis-stack-server-6.2.6-v7/lib/redisearch.so
./redis-stack-server-6.2.6-v7/etc/
./redis-stack-server-6.2.6-v7/etc/README
./redis-stack-server-6.2.6-v7/etc/redis-stack.conf
./redis-stack-server-6.2.6-v7/etc/redis-stack-se

In [26]:
# Add vectors to Redis
from redis import Redis
from redis.commands.search.field import VectorField, TextField
from redis.commands.search.query import Query

# Examples https://github.com/RediSearch/RediSearch/blob/master/docs/docs/vecsim-hybrid_queries_examples.ipynb
# More: https://lablab.ai/t/efficient-vector-similarity-search-with-redis-a-step-by-step-tutorial

# Redis client connection
host = "localhost"
port = 6379

redis_conn = Redis(host=host, port=port)

In [27]:
# Index fields and configurations
# Params  https://redis.io/docs/stack/search/reference/vectors/

n_vec = 10
dim = 384
M = 40  # Optional Number of maximum allowed outgoing edges for each node in the graph in each layer.
EF = 200  # Number of maximum allowed potential outgoing edges candidates for each node in the graph
vector_field = "vector"
token_field_name = "token"
index_name = "viberary"
distance_metric = "COSINE"
float_type = "FLOAT64"

In [28]:
def delete_data(client: Redis):
    client.flushall()

# Delete and rebuild index
delete_data(redis_conn)

# Vectors in redis are stored in a binary blob
def load_docs(client: Redis, n, d, vector_dict):
    # an input dictionary from a dictionary
    for i, (k, v) in enumerate(embedding_dict.items()):
        logging.info(f"Inserting {i} vector into Redis index {index_name}")
        data = np.array(v, dtype=np.float64)
        np_vector = data.astype(np.float64)
        try:
            # try storing vector, log exceptions if fails to map
            client.hset(k, mapping={vector_field: np_vector.tobytes()})
            logging.info(f"Set {k} vector into Redis index as {vector_field}")
        except Exception as e:
            logging.error("An exception occurred: {}".format(e))


def print_results(res):
    docs = [int(doc.id) for doc in res.docs]
    dists = [float(doc.dist) if hasattr(doc, "dist") else "-" for doc in res.docs]
    print(f"got {len(docs)} doc ids: ", docs)
    print("\ndistances: ", dists)



In [29]:
schema = (
    VectorField(
        vector_field,
        "HNSW",
        {"TYPE": "FLOAT64", "DIM": dim, "DISTANCE_METRIC": "COSINE"},
    ),
    TextField(token_field_name),
)
redis_conn.ft(index_name).create_index(schema)
redis_conn.ft(index_name).config_set("default_dialect", 2)

# load vectors with meta-data
load_docs(redis_conn, n_vec, dim, embedding_dict)

INFO:root:Inserting 0 vector into Redis index viberary
INFO:root:Set 0 vector into Redis index as vector
INFO:root:Inserting 1 vector into Redis index viberary
INFO:root:Set 1 vector into Redis index as vector
INFO:root:Inserting 2 vector into Redis index viberary
INFO:root:Set 2 vector into Redis index as vector
INFO:root:Inserting 3 vector into Redis index viberary
INFO:root:Set 3 vector into Redis index as vector
INFO:root:Inserting 4 vector into Redis index viberary
INFO:root:Set 4 vector into Redis index as vector
INFO:root:Inserting 5 vector into Redis index viberary
INFO:root:Set 5 vector into Redis index as vector
INFO:root:Inserting 6 vector into Redis index viberary
INFO:root:Set 6 vector into Redis index as vector
INFO:root:Inserting 7 vector into Redis index viberary
INFO:root:Set 7 vector into Redis index as vector
INFO:root:Inserting 8 vector into Redis index viberary
INFO:root:Set 8 vector into Redis index as vector
INFO:root:Inserting 9 vector into Redis index viberary


In [30]:


print("index meta: ", redis_conn.ft("viberary").info())



index meta:  {'index_name': 'viberary', 'index_options': [], 'index_definition': [b'key_type', b'HASH', b'prefixes', [b''], b'default_score', b'1'], 'attributes': [[b'identifier', b'vector', b'attribute', b'vector', b'type', b'VECTOR'], [b'identifier', b'token', b'attribute', b'token', b'type', b'TEXT', b'WEIGHT', b'1']], 'num_docs': '50', 'max_doc_id': '50', 'num_terms': '0', 'num_records': '50', 'inverted_sz_mb': '0', 'vector_index_sz_mb': '3.1760330200195312', 'total_inverted_index_blocks': '0', 'offset_vectors_sz_mb': '0', 'doc_table_size_mb': '0.0032329559326171875', 'sortable_values_size_mb': '0', 'key_table_size_mb': '0.0013828277587890625', 'records_per_doc_avg': '1', 'bytes_per_record_avg': '0', 'offsets_per_term_avg': '0', 'offset_bits_per_record_avg': '-nan', 'hash_indexing_failures': '0', 'total_indexing_time': '2.6509999999999998', 'indexing': '0', 'percent_indexed': '1', 'number_of_uses': 1, 'gc_stats': [b'bytes_collected', b'0', b'total_ms_run', b'0', b'total_cycles', b'